In [ ]:
import numpy as np
import pandas as pd

from generate_data import load_data
import lifelines
from lifelines import CoxPHFitter
from pycox.datasets import metabric, support

data = 'metabric'
df = metabric.read_df()

time = [min(df['duration'])+i*0.01 for i in range(int((max(df['duration'])-min(df['duration']))/0.01))]

train_frac = 0.8
alpha = 0.95
epochs = 100

alphas = [0.6,0.7,0.8,0.9,0.95]
for alpha in alphas:
    coverage = []
    interval_len = []
    for epoch in range(epochs):
          rng = np.random.RandomState(epoch)
          shuffle_idx = rng.permutation(range(len(df)))
          train_idx = shuffle_idx[:int(train_frac*len(df))]
          test_idx = shuffle_idx[int(train_frac*len(df)):]
          df_train = df.iloc[train_idx,:]
          df_test = df.iloc[test_idx,:]
    #       df_train = df_train.drop(columns=['duration_true'])
    #       duration_true = df_test['duration_true']
    #       df_test = df_test.drop(columns=['duration_true'])
          cph = CoxPHFitter()
          cph.fit(df_train,duration_col = 'duration',event_col = 'event')

          surv = cph.predict_survival_function(df_test.iloc[:,:9],times=time)
          surv_ = (surv<=1-alpha).to_numpy(dtype='int8')
          index = np.array(surv.index)
          multiply_surv = np.transpose(surv_)*index
          multiply_surv_ = np.where(multiply_surv==0,np.max(index),multiply_surv)

          t_predict = multiply_surv_.min(axis = 1)
          diff_predict_true = np.subtract(t_predict,np.array(df_test['duration']))

          cover = sum(diff_predict_true>=0)/len(t_predict)

          coverage.append(cover)
          print('[%d]\t%.3f'%(epoch,cover))
          interval_len.append(np.mean(t_predict))
    print('Total Coverage Statistics:\t [Mean]%.3f\t[Std.]%.3f\t[Max]%.3f\t[Min]%.3f'%(np.mean(coverage),np.std(coverage),np.max(coverage),np.min(coverage)))

    np.savetxt('./output/cox_reg_coverage_'+data+'_'+str(epochs)+str(alpha)+'.txt',np.array(coverage))
    np.savetxt('./output/cox_reg_interval_'+data+'_'+str(epochs)+str(alpha)+'.txt',np.array(interval_len))

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


[0]	0.774
[1]	0.772
[2]	0.761
[3]	0.819
[4]	0.785
[5]	0.769
[6]	0.772
[7]	0.790
[8]	0.761
[9]	0.772
[10]	0.808
[11]	0.835
[12]	0.824
[13]	0.764
[14]	0.787
[15]	0.774
[16]	0.811
[17]	0.814
[18]	0.798
[19]	0.785
[20]	0.777
[21]	0.751
[22]	0.795
[23]	0.814
[24]	0.798
[25]	0.764
[26]	0.766
[27]	0.761
[28]	0.798
[29]	0.769
[30]	0.795
[31]	0.803
[32]	0.764
[33]	0.835
[34]	0.766
[35]	0.803
[36]	0.782
[37]	0.766
[38]	0.782
[39]	0.866
[40]	0.787
[41]	0.782
[42]	0.769
[43]	0.803
[44]	0.808
[45]	0.803
[46]	0.798
[47]	0.822
[48]	0.801
[49]	0.782
[50]	0.835
[51]	0.790
[52]	0.822
[53]	0.806
[54]	0.780
[55]	0.806
[56]	0.819
[57]	0.829
[58]	0.748
[59]	0.772
[60]	0.780
[61]	0.827
[62]	0.774
[63]	0.801
[64]	0.801
[65]	0.806
[66]	0.753
[67]	0.803
[68]	0.801
[69]	0.795
[70]	0.748
[71]	0.782
[72]	0.803
[73]	0.780
[74]	0.816
[75]	0.782
[76]	0.785
[77]	0.787
[78]	0.790
[79]	0.801
[80]	0.814
[81]	0.806
[82]	0.793
[83]	0.848
[84]	0.835
[85]	0.748
[86]	0.759
[87]	0.782
[88]	0.790
[89]	0.777
[90]	0.816
[91]	0.81